In [41]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report

import  yfinance as yf

In [42]:
# Get historical OHLCV data for target ticker
etf_df = yf.download(
    "STW.AX, VAS.AX, SPY.AX", 
    period="max"
)

[*********************100%***********************]  3 of 3 completed


In [43]:
# Review data to see the date ranges
display(etf_df.head())
display(etf_df.tail())
display(etf_df.shape)
display(etf_df.index)


Adj Close                    Close                     High  \
              SPY.AX     STW.AX VAS.AX SPY.AX     STW.AX VAS.AX SPY.AX   
Date                                                                     
2008-01-02       NaN  61.169998    NaN    NaN  61.169998    NaN    NaN   
2008-01-03       NaN  60.590000    NaN    NaN  60.590000    NaN    NaN   
2008-01-04       NaN  60.740002    NaN    NaN  60.740002    NaN    NaN   
2008-01-07       NaN  59.349998    NaN    NaN  59.349998    NaN    NaN   
2008-01-08       NaN  59.020000    NaN    NaN  59.020000    NaN    NaN   

                                Low                     Open             \
               STW.AX VAS.AX SPY.AX     STW.AX VAS.AX SPY.AX     STW.AX   
Date                                                                      
2008-01-02  61.169998    NaN    NaN  61.169998    NaN    NaN  61.169998   
2008-01-03  60.590000    NaN    NaN  60.590000    NaN    NaN  60.590000   
2008-01-04  60.740002    NaN    NaN  60.740002    NaN    NaN  60.740002   
2008-01-07  59.349998    NaN    NaN  59.349998    NaN    NaN  59.349998   
2008-01-08  59.020000    NaN    NaN  59.020000    NaN    NaN  59.020000   

                  Volume                
           VAS.AX SPY.AX STW.AX VAS.AX  
Date                                    
2008-01-02    NaN    NaN      0    NaN  
2008-01-03    NaN    NaN      0    NaN  
2008-01-04    NaN    NaN      0    NaN  
2008-01-07    NaN    NaN      0    NaN  
2008-01-08    NaN    NaN      0    NaN

Adj Close                             Close             \
                SPY.AX     STW.AX     VAS.AX      SPY.AX     STW.AX   
Date                                                                  
2022-12-19  571.799988  64.849998  88.800003  571.799988  64.849998   
2022-12-20  567.669983  63.900002  87.440002  567.669983  63.900002   
2022-12-21  573.729980  64.669998  88.500000  573.729980  64.669998   
2022-12-22  573.000000  65.040001  88.949997  573.000000  65.040001   
2022-12-23  570.000000  64.610001  88.360001  570.000000  64.610001   

                             High                               Low  \
               VAS.AX      SPY.AX     STW.AX     VAS.AX      SPY.AX   
Date                                                                  
2022-12-19  88.800003  574.559998  65.040001  89.050003  571.500000   
2022-12-20  87.440002  569.679993  64.809998  88.709999  562.400024   
2022-12-21  88.500000  574.520020  64.980003  88.839996  570.059998   
2022-12-22  88.949997  578.000000  65.199997  89.180000  573.000000   
2022-12-23  88.360001  573.030029  64.629997  88.389999  569.900024   

                                        Open                       Volume  \
               STW.AX     VAS.AX      SPY.AX     STW.AX     VAS.AX SPY.AX   
Date                                                                        
2022-12-19  64.709999  88.570000  574.559998  64.730003  88.589996  453.0   
2022-12-20  63.799999  87.300003  567.750000  64.809998  88.570000  779.0   
2022-12-21  64.430000  88.129997  570.059998  64.430000  88.139999  435.0   
2022-12-22  65.019997  88.879997  577.330017  65.110001  89.000000  199.0   
2022-12-23  64.220001  87.820000  573.030029  64.610001  88.339996  103.0   

                               
            STW.AX     VAS.AX  
Date                           
2022-12-19   65356   239837.0  
2022-12-20  125904   357111.0  
2022-12-21   69366   154546.0  
2022-12-22   52931  1209246.0  
2022-12-23  110893   207291.0

(3791, 18)

DatetimeIndex(['2008-01-02', '2008-01-03', '2008-01-04', '2008-01-07',
               '2008-01-08', '2008-01-09', '2008-01-10', '2008-01-11',
               '2008-01-14', '2008-01-15',
               ...
               '2022-12-12', '2022-12-13', '2022-12-14', '2022-12-15',
               '2022-12-16', '2022-12-19', '2022-12-20', '2022-12-21',
               '2022-12-22', '2022-12-23'],
              dtype='datetime64[ns]', name='Date', length=3791, freq=None)

In [44]:
# Examine the column type and check for nulls
etf_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3791 entries, 2008-01-02 to 2022-12-23
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   (Adj Close, SPY.AX)  2075 non-null   float64
 1   (Adj Close, STW.AX)  3791 non-null   float64
 2   (Adj Close, VAS.AX)  3454 non-null   float64
 3   (Close, SPY.AX)      2075 non-null   float64
 4   (Close, STW.AX)      3791 non-null   float64
 5   (Close, VAS.AX)      3454 non-null   float64
 6   (High, SPY.AX)       2075 non-null   float64
 7   (High, STW.AX)       3791 non-null   float64
 8   (High, VAS.AX)       3454 non-null   float64
 9   (Low, SPY.AX)        2075 non-null   float64
 10  (Low, STW.AX)        3791 non-null   float64
 11  (Low, VAS.AX)        3454 non-null   float64
 12  (Open, SPY.AX)       2075 non-null   float64
 13  (Open, STW.AX)       3791 non-null   float64
 14  (Open, VAS.AX)       3454 non-null   float64
 15  (Volume, SPY.AX)    

In [45]:
# Retain the key columns for each etf
etf_df = etf_df.drop(columns = ['Adj Close', 'High', 'Low', 'Open', 'Volume'])
etf_df.columns = etf_df.columns.droplevel()     # Changes the multilevel indexing on columns to single level


In [46]:
etf_df.head()

,SPY.AX,STW.AX,VAS.AX
Date,,,
2008-01-02,NaN,61.169998,NaN
2008-01-03,NaN,60.590000,NaN
2008-01-04,NaN,60.740002,NaN
2008-01-07,NaN,59.349998,NaN
2008-01-08,NaN,59.020000,NaN


In [47]:
#  Drop all nulls
etf_df = etf_df.dropna()
etf_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2074 entries, 2014-10-13 to 2022-12-23
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   SPY.AX  2074 non-null   float64
 1   STW.AX  2074 non-null   float64
 2   VAS.AX  2074 non-null   float64
dtypes: float64(3)
memory usage: 64.8 KB


In [48]:
etf_df.head()

,SPY.AX,STW.AX,VAS.AX
Date,,,
2014-10-13,216.729996,48.599998,65.360001
2014-10-14,213.949997,49.119999,66.059998
2014-10-15,217.089996,49.500000,66.480003
2014-10-16,212.434738,49.549999,66.599998
2014-10-17,213.059998,49.779999,66.839996


In [51]:
# Downloading news items related to a specific stock
spy_df = yf.Ticker('SPY.AX')
spy_df.info

{'exchange': 'ASX',
 'shortName': 'SPDR 500 CDI 1:1',
 'longName': 'SPDR S&P 500 ETF Trust',
 'exchangeTimezoneName': 'Australia/Sydney',
 'exchangeTimezoneShortName': 'AEDT',
 'isEsgPopulated': False,
 'gmtOffSetMilliseconds': '39600000',
 'quoteType': 'ETF',
 'symbol': 'SPY.AX',
 'messageBoardId': 'finmb_6160262',
 'market': 'au_market',
 'annualHoldingsTurnover': None,
 'enterpriseToRevenue': None,
 'beta3Year': 0.97,
 'profitMargins': None,
 'enterpriseToEbitda': None,
 '52WeekChange': None,
 'morningStarRiskRating': None,
 'forwardEps': None,
 'revenueQuarterlyGrowth': None,
 'fundInceptionDate': 727660800,
 'annualReportExpenseRatio': None,
 'totalAssets': 134735440,
 'bookValue': None,
 'fundFamily': 'State Street Global Advisors (Aus) Ltd',
 'lastFiscalYearEnd': None,
 'netIncomeToCommon': None,
 'trailingEps': None,
 'lastDividendValue': None,
 'SandP52WeekChange': None,
 'priceToBook': None,
 'nextFiscalYearEnd': None,
 'yield': 0.0114,
 'mostRecentQuarter': None,
 'enterpris

In [52]:
spy_df.news

[{'uuid': '959be3a7-3cd4-3127-9d73-a7edf7ad2d39',
  'title': 'Could The Market Be Wrong About Myer Holdings Limited (ASX:MYR) Given Its Attractive Financial Prospects?',
  'publisher': 'Simply Wall St.',
  'link': 'https://finance.yahoo.com/news/could-market-wrong-myer-holdings-231611805.html',
  'providerPublishTime': 1671923771,
  'type': 'STORY',
  'thumbnail': {'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/8TbV6wt98k2uNVVz1zBXug--~B/aD00MzI7dz0xMTk0O2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/simply_wall_st__316/d72a9b1a079e0dab51654ccf2e73ceec',
     'width': 1194,
     'height': 432,
     'tag': 'original'},
    {'url': 'https://s.yimg.com/uu/api/res/1.2/3LTgR8kP721NKnD7Obaksg--~B/Zmk9ZmlsbDtoPTE0MDtweW9mZj0wO3c9MTQwO2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/simply_wall_st__316/d72a9b1a079e0dab51654ccf2e73ceec',
     'width': 140,
     'height': 140,
     'tag': '140x140'}]},
  'relatedTickers': ['MYR.AX']},
 {'uuid': '304d49c1-08f6-3802-9641-adfbdaae417d',
 

In [53]:
len(spy_df.news)

8

In [54]:
spy_df.news[-1]

{'uuid': '79085fb5-0237-3259-ae64-66eb0ead0f97',
 'title': 'How Rich Is Meghan Markle?',
 'publisher': 'GOBankingRates',
 'link': 'https://finance.yahoo.com/news/rich-harry-meghan-200058201.html',
 'providerPublishTime': 1671922892,
 'type': 'STORY',
 'thumbnail': {'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/3kOlcSlua7tGVvwCqapbyg--~B/aD0xMDgwO3c9MTkyMDthcHBpZD15dGFjaHlvbg--/https://media.zenfs.com/en/gobankingrates_644/c4958cd3419a1b0fb3705c307a0dd8d7',
    'width': 1920,
    'height': 1080,
    'tag': 'original'},
   {'url': 'https://s.yimg.com/uu/api/res/1.2/pn4ti1w1r11sUKXeyX.6pQ--~B/Zmk9ZmlsbDtoPTE0MDtweW9mZj0wO3c9MTQwO2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/gobankingrates_644/c4958cd3419a1b0fb3705c307a0dd8d7',
    'width': 140,
    'height': 140,
    'tag': '140x140'}]}}